In [1]:
"""
python3 main.py ../../dataset/train.csv ../../dataset/test.csv ../result/cv_results.csv ../result/submission.csv > ../result/logs.txt

make train

"""
import sys
sys.path.append("../fraud_detection/src/")
import time
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from contextlib import contextmanager
import gc 
from util import s_to_time_format, string_to_datetime, hour_to_range, kfold_lightgbm, kfold_xgb
from util import _time_elapsed_between_last_transactions,time_elapsed_between_last_transactions
from util import num_transaction_in_past_n_days
from time import strftime, localtime
import logging
import sys
from config import Configs
from extraction import merge_and_split_dfs, get_conam_dict_by_day, last_x_day_conam

# logging
logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# logger.addHandler(logging.StreamHandler(sys.stdout))
# #log_file = '{}-{}-{}.log'.format(opt.model_name, opt.dataset, strftime("%y%m%d-%H%M", localtime()))
# log_file = '../result/{}.log'.format(strftime("%y%m%d-%H%M", localtime()))
# logger.addHandler(logging.FileHandler(log_file))

def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            df_test = df_test.merge(tmp, how='left', on=cols)
            
            del tmp
            gc.collect()
    
    return df_train, df_test

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))
    
def main(args):
    with timer("Process train/test application"):
        #-------------------------
        # load dataset
        #-------------------------
        df_train = pd.read_csv(args.train_file)
        df_test = pd.read_csv(args.test_file)

        #-------------------------
        # pre-processing
        #-------------------------

        for cat in Configs.CATEGORY:
            df_train[cat] = df_train[cat].astype('category') #.cat.codes
            df_test[cat] = df_test[cat].astype('category')
            
        for df in [df_train, df_test]:
            # pre-processing
            df["loctm_"] = df.loctm.astype(int).astype(str)
            df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
            # # time-related feature
            df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour).astype('category')
            df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
            df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)
            # df["loctm_absolute_time"] = [h*60+m for h,m in zip(df.loctm_hour_of_day,df.loctm_minute_of_hour)]
            df["hour_range"] = df.loctm_.apply(lambda x: hour_to_range(x.hour)).astype("category")
            # removed the columns no need
            df.drop(columns = ["loctm_"], axis = 1, inplace = True)
        print("Train application df shape: {}".format(df_train.shape))
        print("Test application df shape: {}".format(df_test.shape))
        
#     with timer('Add time-aggregate features'):

#         df, split_df = merge_and_split_dfs(df_train, df_test)
#         conam_dict = get_conam_dict_by_day(df)

#         df['last_3_day_mean_conam_per_day'] = last_x_day_conam(3, df, conam_dict)
#         df['last_7_day_mean_conam_per_day'] = last_x_day_conam(7, df, conam_dict)
#         df['last_10_day_mean_conam_per_day'] = last_x_day_conam(10, df, conam_dict)
#         df['last_14_day_mean_conam_per_day'] = last_x_day_conam(14, df, conam_dict)
#         df['last_30_day_mean_conam_per_day'] = last_x_day_conam(30, df, conam_dict)
#         df['last_45_day_mean_conam_per_day'] = last_x_day_conam(45, df, conam_dict)
#         df['last_60_day_mean_conam_per_day'] = last_x_day_conam(60, df, conam_dict)

#         df_train, df_test = split_df(df)
#         print("Train application df shape: {}".format(df_train.shape))
#         print("Test application df shape: {}".format(df_test.shape))

    with timer("Add bacno/cano feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_1)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add iterm-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.ITERM_AGG_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add conam-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_2)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add hour-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.HOUR_AGG_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add cano/conam feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CANO_CONAM_COUNT_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add cano/bacno latent feature"):
#         df = pd.read_csv("../fraud_detection/features/bacno_latent_features.csv")
#         df_train = df_train.merge(df, on = "bacno", how = "left")
#         df_test = df_test.merge(df, on = "bacno", how = "left")
#         df = pd.read_csv("../fraud_detection/features/cano_latent_features.csv")
#         df_train = df_train.merge(df, on = "cano", how = "left")
#         df_test = df_test.merge(df, on = "cano", how = "left")

#         print("Train application df shape: {}".format(df_train.shape))
#         print("Test application df shape: {}".format(df_test.shape))

#     with timer("Add locdt-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.LOCDT_CONAM_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add mchno-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.MCHNO_CONAM_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add scity-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.SCITY_CONAM_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add stocn-related feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.STOCN_CONAM_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add mchno/bacno latent feature"):
#         df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_mchno.csv")
#         df_train = df_train.merge(df, on = "bacno", how = "left")
#         df_test = df_test.merge(df, on = "bacno", how = "left")
#         df = pd.read_csv("../fraud_detection/features/mchno_latent_features.csv")
#         df_train = df_train.merge(df, on = "mchno", how = "left")
#         df_test = df_test.merge(df, on = "mchno", how = "left")

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add scity/bacno latent feature"):
#         df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_scity.csv")
#         df_train = df_train.merge(df, on = "bacno", how = "left")
#         df_test = df_test.merge(df, on = "bacno", how = "left")
#         df = pd.read_csv("../fraud_detection/features/scity_latent_features.csv")
#         df_train = df_train.merge(df, on = "scity", how = "left")
#         df_test = df_test.merge(df, on = "scity", how = "left")

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

#     with timer("Add stocn/bacno latent feature"):
#         df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_stocn.csv")
#         df_train = df_train.merge(df, on = "bacno", how = "left")
#         df_test = df_test.merge(df, on = "bacno", how = "left")
#         df = pd.read_csv("../fraud_detection/features/stocn_latent_features.csv")
#         df_train = df_train.merge(df, on = "stocn", how = "left")
#         df_test = df_test.merge(df, on = "stocn", how = "left")

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))
        
#     with timer("Add elapsed time feature"):
#         df = pd.concat([df_train, df_test], axis = 0)
#         df.sort_values(by = ["bacno","locdt"], inplace = True)
        
#         df["time_elapsed_between_last_transactions"] = df[["bacno","locdt"]] \
#         .groupby("bacno").apply(_time_elapsed_between_last_transactions).values
        
#         df_train = df[~df.fraud_ind.isnull()]
#         df_test = df[df.fraud_ind.isnull()]
        
#         df_test.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
#         del df
#         gc.collect()

#         df_train["time_elapsed_between_last_transactions"] = df_train[["bacno","locdt","time_elapsed_between_last_transactions"]] \
#         .groupby(["bacno","locdt"]).apply(time_elapsed_between_last_transactions).values
        
#         df_test["time_elapsed_between_last_transactions"] = df_test[["bacno","locdt","time_elapsed_between_last_transactions"]] \
#         .groupby(["bacno","locdt"]).apply(time_elapsed_between_last_transactions).values
        
#         print("Train application df shape: {}".format(df_train.shape))
#         print("Test application df shape: {}".format(df_test.shape))

#     with timer("Add historical-related feature"):
#         df = pd.concat([df_train, df_test], axis = 0)
#         df.sort_values(by = ["bacno","locdt"], inplace = True)
        
#         for past_n_days in [2,3,4,5,6,7,14,30]:
#             df["num_transaction_in_past_{}_days".format(past_n_days)] = df[["bacno","locdt"]].groupby("bacno")\
#             .apply(lambda x: num_transaction_in_past_n_days(x,past_n_days)).values

#         df_train = df[~df.fraud_ind.isnull()]
#         df_test = df[df.fraud_ind.isnull()]
        
#         df_test.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
#         del df
#         gc.collect()
       
#         print("Train application df shape: {}".format(df_train.shape))
#         print("Test application df shape: {}".format(df_test.shape))


#     with timer("Add elapsed time agg feature"):
#         df_train, df_test = group_target_by_cols(df_train, df_test, Configs.TIME_ELAPSED_AGG_RECIPE)

#         logger.info("Train application df shape: {}".format(df_train.shape))
#         logger.info("Test application df shape: {}".format(df_test.shape))

    return df_train, df_test


In [2]:
args = {
 "train_file":"/data/yunrui_li/fraud/dataset/train.csv",
 "test_file":"/data/yunrui_li/fraud/dataset/test.csv",
 "result_path":"/data/yunrui_li/fraud/fraud_detection/result/submission.csv",
 "feature_selection":False,
 "feature_importance_plot": True,
 "SEED": 1030,
 "NUM_FOLDS": 2, # 5
 "CPU_USE_RATE":1.0,
 "STRATIFIED": True,
 "TEST_NULL_HYPO":False,
 "NUM_LEAVES":31,
 "COLSAMPLE_BYTREE":1.0,
 "SUBSAMPLE": 1.0,
 "SUBSAMPLE_FREQ": 0,
 "MAX_DEPTH": -1,
 "REG_ALPHA": 0.0,
 "REG_LAMBDA": 0.0,
 "MIN_SPLIT_GAIN": 0.0,
 "MIN_CHILD_WEIGHT": 0.001,
 "MAX_BIN": 255,
 "SCALE_POS_WEIGHT": 3
    
}

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
args = AttrDict(args)
df_train, df_test = main(args)


Train application df shape: (1521787, 27)
Test application df shape: (421665, 26)
Process train/test application - done in 68s


/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Add bacno/cano feature - done in 16s


In [3]:
df_train.shape

(1521787, 41)

In [ ]:
#10/8

In [66]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 500

df_test

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,hour_range,count_conam_BY_cano,min_conam_BY_cano,max_conam_BY_cano,mean_conam_BY_cano,median_conam_BY_cano,var_conam_BY_cano,sum_conam_BY_cano,count_conam_BY_bacno_cano,min_conam_BY_bacno_cano,max_conam_BY_bacno_cano,mean_conam_BY_bacno_cano,median_conam_BY_bacno_cano,var_conam_BY_bacno_cano,sum_conam_BY_bacno_cano
182946,6189,0,114775,929.46,5,62,N,2,N,N,5,N,0,102,100045.0,343,87081,N,5817,102,0,1751253,10,0,45,morning,1,929.46,929.46,929.460000,929.460,NaN,929.46,1,929.46,929.46,929.460000,929.460,NaN,929.46
412770,5975,5,49266,225.84,5,62,N,4,N,N,5,N,0,119,101355.0,263,92577,N,5817,102,0,1688453,10,13,55,morning,1,225.84,225.84,225.840000,225.840,NaN,225.84,1,225.84,225.84,225.840000,225.840,NaN,225.84
97049,5112,6,213186,513.80,5,0,N,0,N,N,5,N,0,99,72127.0,453,544,N,0,102,0,635437,7,21,27,early_morning,2,210.36,513.80,362.080000,362.080,46037.916800,724.16,2,210.36,513.80,362.080000,362.080,46037.916800,724.16
97048,6769,6,213186,210.36,5,62,N,5,N,N,5,N,0,113,202621.0,251,84157,N,5817,102,0,417354,20,26,21,night,2,210.36,513.80,362.080000,362.080,46037.916800,724.16,2,210.36,513.80,362.080000,362.080,46037.916800,724.16
223371,5982,10,59511,419.79,5,62,N,4,N,N,5,N,0,91,151539.0,251,24881,N,3297,102,0,888505,15,15,39,afternoon,3,216.12,844.77,493.560000,419.790,102881.715300,1480.68,3,216.12,844.77,493.560000,419.790,102881.715300,1480.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299284,6215,163885,178691,0.00,5,62,N,2,N,N,5,N,0,96,154656.0,289,54828,N,3460,46,0,1932816,15,46,56,afternoon,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56
299278,6215,163885,178691,168.20,5,62,Y,2,N,N,5,N,0,106,154832.0,289,54828,N,3460,46,0,1941526,15,48,32,afternoon,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56
299281,6557,163885,178691,1608.36,5,62,N,5,N,N,5,N,0,108,141150.0,288,31653,N,5817,102,0,1040128,14,11,50,noon,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56,3,0.00,1608.36,592.186667,168.200,781528.992533,1776.56
148335,6769,163886,42854,2403.67,4,62,N,2,N,N,5,N,0,109,90150.0,343,78638,N,5817,102,0,300674,9,1,50,morning,2,2403.67,2432.74,2418.205000,2418.205,422.532450,4836.41,2,2403.67,2432.74,2418.205000,2418.205,422.532450,4836.41


In [67]:
df_test.groupby("cano").conam.count()

cano
6          9
12         3
13         5
15         1
16        11
          ..
213569     3
213570     1
213571     2
213572     1
213575     1
Name: conam, Length: 86808, dtype: int64

In [74]:
df_test.bacno.sample()

179746    158830
Name: bacno, dtype: int64

In [23]:
df_test.sort_values(by = ["bacno","cano",
                          "locdt","loctm_hour_of_day",
                          "loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

In [76]:
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 500

df_test[df_test.bacno == 158830][["bacno","cano","locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"]]

,bacno,cano,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min
179204,158830,37028,104,15,7,46
179283,158830,37028,104,16,0,24
180115,158830,37028,104,16,2,23
179616,158830,37028,104,16,3,58
179625,158830,37028,104,16,5,39
...,...,...,...,...,...,...
179401,158830,37028,120,17,25,52
179745,158830,37028,120,17,25,57
179391,158830,37028,120,17,27,40
179914,158830,37028,120,17,27,45


In [28]:
df_train.sort_values(by = ["bacno","cano",
                          "locdt","loctm_hour_of_day",
                          "loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

In [47]:
fraudulent_caro = set(df_train[df_train.fraud_ind==1].cano.unique().tolist())

In [49]:
# normal transaction
for cano, row in df_train[df_train.cano.isin(fraudulent_caro)].head(100).groupby("cano"):
    print (cano)
    print (row.locdt.tolist())
    print (row.conam.tolist())
    print (row.fraud_ind.tolist())

14264
[4, 6, 6, 8, 11]
[611.33, 674.26, 300.32, 600.67, 281.38]
[0, 0, 0, 0, 0]
44120
[7, 26, 33]
[103.48, 713.91, 713.91]
[0, 1, 1]
46159
[18, 18, 24, 24, 24, 55]
[1588.74, 1152.73, 578.02, 1202.35, 683.25, 0.0]
[0, 0, 0, 0, 0, 1]
61182
[2]
[2346.74]
[1]
78063
[18, 20, 29, 31, 31, 33, 37, 37, 39, 43, 45, 56, 57, 60, 62, 62, 62, 62, 62, 62, 62, 62, 62]
[756.62, 465.62, 513.8, 967.28, 941.28, 513.8, 903.34, 722.38, 760.74, 555.04, 0.0, 796.9, 560.84, 625.04, 1517.71, 948.73, 948.73, 948.73, 948.73, 948.61, 948.61, 948.29, 948.29]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
90488
[5, 13, 20, 23, 30, 30, 30, 31, 31, 36, 40]
[1918.3, 465.62, 465.62, 465.62, 0.0, 0.0, 168.2, 0.0, 0.0, 465.62, 168.2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
99027
[3, 3, 11, 12, 38, 41, 61, 64, 64, 64, 65, 65, 65, 65, 76, 76]
[526.88, 554.65, 564.28, 547.93, 665.75, 2123.56, 542.29, 0.0, 943.95, 943.99, 532.8, 529.43, 944.96, 944.96, 952.56, 952.56]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0

In [41]:
df_train[df_train.cano == 96923][["bacno","cano",
                          "locdt","loctm_hour_of_day",
                          "loctm_minute_of_hour","loctm_second_of_min"]]

,bacno,cano,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min
1278126,2,96923,1,9,22,23
481299,2,96923,1,9,46,39
263679,2,96923,1,18,9,17
559276,2,96923,2,13,34,51
1518063,2,96923,2,13,50,33
...,...,...,...,...,...,...
325775,2,96923,85,16,57,35
591517,2,96923,86,9,28,47
1503882,2,96923,89,9,21,28
1283546,2,96923,89,10,28,23


In [49]:
pd.options.display.max_rows = 500

test_case = df_train[df_train.bacno.isin([1,2,113261,134508])][["bacno","cano","conam","fraud_ind","locdt",
                                                      "loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"]]

In [92]:
# test_case.sort_values(by = ["bacno","locdt","loctm_hour_of_day",
#                             "loctm_minute_of_hour","loctm_second_of_min","loctm_second_of_min"], inplace = True)
len(test_case)
test_case

,bacno,cano,conam,fraud_ind,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min
0,113261,38038,513.80,0,33,17,26,52
1,134508,45725,465.62,0,9,10,51,14
10441,1,117264,613.81,0,34,15,5,12
14351,2,101554,0.00,0,32,9,49,50
16628,113261,38038,612.50,0,34,21,6,3
18097,113261,38038,513.80,0,33,17,26,13
27691,113261,38038,1467.41,0,49,23,2,46
28108,2,96923,303.93,0,44,10,30,24
34354,113261,38038,403.35,0,49,20,31,40
38006,2,96923,242.80,0,85,10,32,38


In [135]:
Configs.HOUR_AGG_SEC_LEVEL_RECIPE[0]

(['bacno', 'locdt', 'loctm_hour_of_day', 'loctm_minute_of_hour'],
 [('conam', 'count'),
  ('conam', 'min'),
  ('conam', 'max'),
  ('conam', 'mean'),
  ('conam', 'median'),
  ('conam', 'var'),
  ('conam', 'sum')])

In [6]:
from config import Configs

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    out = []
    for i in range(0, len(l), n):
        out.append(l[i:i + n])
    return out

def group_target_by_cols_split_by_users(df_train, df_test, recipe, user_column = "bacno", num_partitions = 10):
    """
    for memory problem
    """
    df = pd.concat([df_train, df_test], axis = 0)
    print (df.shape)
    df_users = df[user_column].unique().tolist()
    train_users = df_train[user_column].unique().tolist()
    test_users = df_test[user_column].unique().tolist()      
    o_feats = df.columns.tolist()
    
    output = []
    for partition_users in chunks(df_users,num_partitions):
        print ("partition_users",partition_users)
        tmp_df_ls = [] # including partiton of the users w new features we just created
        c = 0
        for m in range(len(recipe)):
            cols = recipe[m][0]
            for n in range(len(recipe[m][1])):
                target = recipe[m][1][n][0]
                method = recipe[m][1][n][1]
                name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
                df_split_by_users = df[df[user_column].isin(partition_users)]
                tmp = df_split_by_users[cols + [target]].groupby(cols).agg(method)
                tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
                if c!= 0:
                    tmp_df_ls.append(df_split_by_users.merge(tmp, how='left', on=cols)[name_grouped_target])
                else:
                    tmp_df_ls.append(df_split_by_users.merge(tmp, how='left', on=cols))
                c+=1
                # reduce memory
                del df_split_by_users,tmp
                gc.collect()             
        tmp_df_ls = pd.concat(tmp_df_ls, axis = 1)
        output.append(tmp_df_ls)
        # reduce memory
        del tmp_df_ls
        gc.collect()             

    df = pd.concat(output, axis = 0) # including all the users w new features we just created

    df_train = df[df[user_column].isin(train_users)]
    df_test = df[df[user_column].isin(test_users)]
    df_test.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
    del df
    gc.collect()  
    return df_train, df_test

# def group_target_by_cols_split_by_users(df_train, df_test, recipe, user_column = "bacno", num_partitions = 2):
#     """
#     for memory problem
#     """
#     df = pd.concat([df_train, df_test], axis = 0)
    
#     df_users = df[user_column].unique().tolist()
#     train_users = df_train[user_column].unique().tolist()
#     test_users = df_test[user_column].unique().tolist()      
    
#     result = []
#     for m in range(len(recipe)):
#         cols = recipe[m][0]
#         for n in range(len(recipe[m][1])):
#             output = [] # including all user w one new feature we just created
#             for partition_users in chunks(df_users,num_partitions):
#                 target = recipe[m][1][n][0]
#                 method = recipe[m][1][n][1]
#                 name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
#                 df_split_by_users = df[df[user_column].isin(partition_users)]
#                 tmp = df_split_by_users[cols + [target]].groupby(cols).agg(method)
#                 tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
#                 # reduce memory
#                 del df_split_by_users,tmp
#                 gc.collect()
#                 output.append(df_split_by_users.merge(tmp, how='left', on=cols))
#             output = pd.concat(output, axis = 0)
 
        
#     df = pd.concat(output, axis = 0)
#     print (len(df))
#     return df
#     df_train = df[df[user_column].isin(train_users)]
#     df_test = df[df[user_column].isin(test_users)]

#     return df_train, df_test

# df_split_by_users,tmp = group_target_by_cols_split_by_users(
#     df_train[df_train.bacno.isin([22313,1])], 
#     df_test[df_test.bacno == 14949],
#     [Configs.HOUR_AGG_SEC_LEVEL_RECIPE[0]],
#     user_column = "bacno",
# )
df_train_, df_test_ = group_target_by_cols_split_by_users(
    df_train[df_train.bacno.isin([22313,1])], 
    df_test[df_test.bacno == 14949],
    Configs.HOUR_AGG_SEC_LEVEL_RECIPE_BACNO,
    user_column = "bacno",
)

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(345, 39)
partition_users [1, 22313]
partition_users [14949]
(330, 53)
(15, 53)
(345, 53)


In [13]:
df_train_.shape, df_test_.shape

((330, 53), (15, 53))

In [16]:
df_train_.columns

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'hour_range', 'insfg',
       'iterm', 'locdt', 'loctm', 'loctm_hour_of_day', 'loctm_minute_of_hour',
       'loctm_second_of_min', 'max_conam_BY_bacno_cano', 'max_conam_BY_cano',
       'mcc', 'mchno', 'mean_conam_BY_bacno_cano', 'mean_conam_BY_cano',
       'median_conam_BY_bacno_cano', 'median_conam_BY_cano',
       'min_conam_BY_bacno_cano', 'min_conam_BY_cano', 'ovrlt', 'scity',
       'stocn', 'stscd', 'sum_conam_BY_bacno_cano', 'sum_conam_BY_cano',
       'txkey', 'var_conam_BY_bacno_cano', 'var_conam_BY_cano',
       'count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour',
       'min_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour',
       'max_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour',
       'mean_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour',
       'median_conam_BY_bacno_locdt_loctm_hour_of

In [17]:
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 500


df_train_\
.sort_values(by = ["bacno","locdt","loctm_hour_of_day",
                            "loctm_minute_of_hour","loctm_second_of_min","loctm_second_of_min"], inplace = True)

df_train_[df_train_.bacno == 22313][["bacno","cano",
           "conam","fraud_ind","locdt", 
           "loctm_hour_of_day","loctm_minute_of_hour",
           "loctm_second_of_min","count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour",
        "count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min"]]


,bacno,cano,conam,fraud_ind,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min
66,22313,164797,103.48,0.0,5,1,58,37,1.0,1.0
13,22313,164796,465.62,0.0,10,10,44,45,1.0,1.0
110,22313,164797,468.74,0.0,14,5,50,11,5.0,2.0
126,22313,164797,467.70,0.0,14,5,50,11,5.0,2.0
295,22313,164797,473.87,0.0,14,5,50,12,5.0,1.0
53,22313,164797,519.10,0.0,14,5,50,13,5.0,2.0
65,22313,164797,492.59,0.0,14,5,50,13,5.0,2.0
153,22313,164797,576.19,0.0,14,6,20,51,7.0,2.0
298,22313,164797,701.45,0.0,14,6,20,51,7.0,2.0
129,22313,164797,701.45,0.0,14,6,20,52,7.0,4.0


In [18]:
df_train_, df_test_ = group_target_by_cols_split_by_users(
    df_train[df_train.bacno.isin([22313,1])], 
    df_test[df_test.bacno == 14949],
    Configs.HOUR_AGG_SEC_LEVEL_RECIPE_CANO,
    user_column = "cano",
)

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(345, 39)
partition_users [117264, 164797]
partition_users [164798, 164796]
partition_users [191028, 70220]
partition_users [70221]
(221, 53)
(109, 53)
(14, 53)
(1, 53)
(345, 53)


In [31]:
df_train_\
.sort_values(by = ["cano","locdt","loctm_hour_of_day",
                            "loctm_minute_of_hour","loctm_second_of_min","loctm_second_of_min"], inplace = True)

df_train_[df_train_.bacno == 22313][["bacno","cano",
           "conam","fraud_ind","locdt", 
           "loctm_hour_of_day","loctm_minute_of_hour",
           "loctm_second_of_min","count_conam_BY_cano_locdt_loctm_hour_of_day_loctm_minute_of_hour",
        "count_conam_BY_cano_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min"]]


,bacno,cano,conam,fraud_ind,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,count_conam_BY_cano_locdt_loctm_hour_of_day_loctm_minute_of_hour,count_conam_BY_cano_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min
2,22313,164796,465.62,0.0,10,10,44,45,1.0,1.0
6,22313,164796,465.62,0.0,25,10,59,13,1.0,1.0
42,22313,164796,1979.62,0.0,32,23,39,2,1.0,1.0
81,22313,164796,465.62,0.0,33,10,58,24,1.0,1.0
82,22313,164796,465.62,0.0,40,10,51,27,1.0,1.0
97,22313,164796,465.62,0.0,43,10,53,19,1.0,1.0
78,22313,164796,465.62,0.0,53,10,17,56,1.0,1.0
61,22313,164796,465.62,0.0,81,10,17,49,1.0,1.0
39,22313,164796,465.62,0.0,87,10,14,44,1.0,1.0
45,22313,164797,103.48,0.0,5,1,58,37,1.0,1.0


In [33]:
df_train.bacno.nunique() /10

9521.4

In [30]:
df_train.cano.nunique() /10

12941.3

In [29]:
df_train.mchno.nunique() /20

4465.8

# 10/6

In [ ]:
def count_before_n_secs(df, n = 3):
    """
    calculate how many transactions happen before n secs.
    """
    output = []
    if len(df)== 1:
        output.append(0)
    else:
        events = df.loctm_second_of_min.tolist()
        for i, e_time_in_sec in enumerate(events):
            history_sec = e_time_in_sec - n
            if i==0 :
                output.append(0)
            else:
                if events[i] >= history_sec:
                    output.append(1)
                else:
                    output.append(0)
    return pd.Series(output)
    #return df
test_case["num_transaction_before_3_secs"] = \
test_case[["bacno","locdt","conam","loctm_minute_of_hour","loctm_hour_of_day","loctm_second_of_min"]]\
.groupby(["bacno","locdt","loctm_hour_of_day","loctm_minute_of_hour"]).apply(count_before_n_secs).values

In [30]:
len(test_case)
test_case = test_case.iloc[:3]
len(test_case)


3

In [38]:
cols = ["bacno","locdt",
        "loctm_hour_of_day",
        "loctm_minute_of_hour"
       ]
# tmp = test_case[cols + ["conam"]].groupby(cols).agg("mean")
# tmp = tmp.reset_index()

In [25]:
# test_case.groupby(cols)[target].rolling(window=window).mean().values
tmp.conam.value_counts()

1979.62    1
613.81     1
1312.92    1
Name: conam, dtype: int64

In [ ]:
HOUR_AGG_SEC_LEVEL_RECIPE = [
    (["bacno","locdt","loctm_hour_of_day","loctm_minute_of_hour"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一歸戶, 在同一天, 同一分鐘, 刷了幾次卡, 刷卡最大金額, ...
    (["bacno","locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一歸戶, 在同一天, 同一分鐘, 同一秒鐘, 刷了幾次卡, 刷卡最大金額, ...
    (["cano","locdt","loctm_hour_of_day","loctm_minute_of_hour"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一卡號, 在同一天, 同一分鐘, 刷了幾次卡, 刷卡最大金額, ...
    (["cano","locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一卡號, 在同一天, 同一分鐘, 同一秒鐘, 刷了幾次卡, 刷卡最大金額, ...
    (["mchno","locdt","loctm_hour_of_day","loctm_minute_of_hour"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一店家, 在同一天, 同一分鐘, 刷了幾次卡, 刷卡最大金額, ...
    (["mchno","locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], [
            ('conam', 'count'),
            ('conam', 'min'),
            ('conam', 'max'),
            ('conam', 'mean'),
            ('conam', 'median'),
            ('conam', 'var'),
            ('conam', 'sum'),
        ]), # 同一店家, 在同一天, 同一分鐘, 同一秒鐘, 刷了幾次卡, 刷卡最大金額, ...

]

def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df
                    _train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            df_test = df_test.merge(tmp, how='left', on=cols)
    del tmp
    gc.collect()
    
    return df_train, df_test
df_train_, df_test_ = group_target_by_cols(df_train[df_train.bacno.isin([22313,1])], df_test[df_test.bacno == 14949],HOUR_AGG_SEC_LEVEL_RECIPE)

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
df_train_, df_test_

In [56]:
df_train_[cols+["loctm_second_of_min",
                "count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour","count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min"]]\
.sort_values(by = ["bacno","locdt","loctm_hour_of_day",
"loctm_minute_of_hour","loctm_second_of_min","loctm_second_of_min"])

,bacno,locdt,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min
108,1,3,20,0,0,1.0,1.0
221,1,4,22,14,28,1.0,1.0
139,1,25,21,26,35,1.0,1.0
301,1,30,20,9,47,1.0,1.0
0,1,34,15,5,12,1.0,1.0
143,1,50,14,48,17,1.0,1.0
142,1,60,13,47,34,1.0,1.0
279,1,63,21,37,23,1.0,1.0
246,1,68,19,29,33,1.0,1.0
67,1,68,23,20,29,1.0,1.0


In [53]:
df_train_

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,loctm_second_of_min,hour_range,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,min_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,max_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,mean_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,median_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,var_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,sum_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour,count_conam_BY_bacno_locdt_loctm_hour_of_day_loctm_minute_of_hour_loctm_second_of_min
0,6189,1,117264,613.81,5,62,N,4,N,N,...,12,afternoon,1.0,613.81,613.81,613.810000,613.810,NaN,613.81,1.0
1,6767,22313,164797,1312.92,5,62,N,4,N,N,...,32,noon,1.0,1312.92,1312.92,1312.920000,1312.920,NaN,1312.92,1.0
2,5575,22313,164797,1979.62,5,62,Y,8,N,N,...,52,night,3.0,1979.62,1979.62,1979.620000,1979.620,0.000000,5938.86,1.0
3,6716,22313,164797,615.44,5,62,Y,8,N,N,...,56,morning,3.0,316.10,836.72,589.420000,615.440,68269.076400,1768.26,3.0
4,6716,22313,164797,1050.29,5,62,Y,8,N,N,...,40,night,1.0,1050.29,1050.29,1050.290000,1050.290,NaN,1050.29,1.0
5,6716,22313,164798,216.12,5,62,Y,8,N,N,...,54,night,12.0,123.24,445.03,246.620000,223.000,13917.323964,2959.44,1.0
6,6716,22313,164797,891.18,5,62,Y,8,N,N,...,11,morning,2.0,827.05,891.18,859.115000,859.115,2056.328450,1718.23,2.0
7,6716,22313,164797,812.26,5,62,Y,8,N,N,...,5,afternoon,2.0,812.26,823.40,817.830000,817.830,62.049800,1635.66,2.0
8,6716,22313,164797,196.76,5,62,Y,8,N,N,...,50,afternoon,11.0,113.65,437.06,247.160000,232.520,9694.606380,2718.76,4.0
9,6716,22313,164797,467.70,5,62,Y,8,N,N,...,15,night,3.0,467.70,780.48,649.453333,700.180,26387.728133,1948.36,2.0


In [ ]:
test_case["mean_conam_in_past_2_transactions_by_bacno"] = test_case.groupby("bacno") \
.conam.rolling(window=2).mean().values


In [ ]:
test_case["mean_conam_in_past_2_transactions_by_cano"] = test_case.groupby("cano") \
.conam.rolling(window=2).mean().values

In [ ]:
test_case[test_case.fraud_ind == 1]

In [ ]:
test_case

In [ ]:
def rolling_stats_target_by_cols(df_train, df_test, recipe, window = 2):
    df = pd.concat([df_train, df_test], axis = 0)
    #df.sort_values(by = ["bacno","locdt"], inplace = True)

    for m in range(len(recipe)):
        cols = recipe[m][0]
        #print (cols + ["locdt"])
        df.sort_values(by = cols + ["locdt"], inplace = True)
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)+"_"+"in_past_{}_transactions".format(window)
            #print (name_grouped_target)
            if method == "mean":
                df[name_grouped_target] = df.groupby(cols)[target].rolling(window=window).mean().values

    df_train = df[~df.fraud_ind.isnull()]
    df_test = df[df.fraud_ind.isnull()]

    df_test.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
    del df
    gc.collect()

    return df_train, df_test

In [ ]:
import gc
recipe = [
        (["mean_conam_in_past_2_transactions_by_cano"], [
            ('conam', 'mean'),
        ]), 
        (["cano"], [
            ('conam', 'mean'),
        ]), 
]

def rolling_stats_target_by_cols(df_train, df_test, recipe, window = 2):
    df = pd.concat([df_train, df_test], axis = 0)
    #df.sort_values(by = ["bacno","locdt"], inplace = True)

    for m in range(len(recipe)):
        cols = recipe[m][0]
        #print (cols + ["locdt"])
        df.sort_values(by = cols + ["locdt"], inplace = True)
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)+"_"+"in_past_{}_transactions".format(window)
            #print (name_grouped_target)
            if method == "mean":
                df[name_grouped_target] = df.groupby(cols)[target].rolling(window=window).mean().values

    df_train = df[~df.fraud_ind.isnull()]
    df_test = df[df.fraud_ind.isnull()]

    df_test.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
    del df
    gc.collect()

    return df_train, df_test

In [ ]:
["a"]+["b"]

In [ ]:
df_train, df_test = rolling_stats_target_by_cols(df_train, df_test, recipe)

In [ ]:
df_train[["bacno","cano","conam","fraud_ind","locdt","mean_conam_BY_bacno_in_past_2_transactions"]]

In [ ]:
df_train[df_train.bacno == 127123][["bacno","cano","conam","fraud_ind","locdt","mean_conam_BY_bacno_in_past_2_transactions","mean_conam_BY_cano_in_past_2_transactions"]]

In [ ]:
df_train[df_train.bacno == 147498][["bacno","cano","conam","fraud_ind","locdt","mean_conam_BY_bacno_in_past_2_transactions"]]

In [ ]:
df_train.shape

In [ ]:
df_test.shape